In [78]:
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

import pandas as pd
from matplotlib import pyplot as plt
import random
import models
import gdown

from scipy.stats import nbinom,norm
from scipy.integrate import trapezoid as trapz

In [2]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTltw5xQVB_sSBCLiA5nzbiDc1srkInw5TDBWcYy50A-Yhr7zKMTgJUy0aoE1q0uCo5WUJSJSVhR_SY/pub?gid=0&single=true&output=csv"
df = pd.read_csv(url)

In [3]:
#file_path = 'results.ods'

# Read the Excel file
#df = pd.read_excel(file_path, engine="odf")

# Print the DataFrame
print(df)

winners=np.array(df['Winner'])
losers=np.array(df['Loser'])
loser_scores=np.array(df['Points'])
names = pd.concat([df['Winner'], df['Loser']]).unique()
ids = list(range(len(names)))
dict_to_id = dict(zip(names,ids))
dict_to_name=dict(zip(ids,names))
winner_ids = [dict_to_id[i] for i in winners]
loser_ids = [dict_to_id[i] for i in losers]

   Winner  Loser  Points
0    Fedo  Mauro       1
1    Fedo  Idris       3
2   Mauro  Idris       6
3    Fedo  Idris       5
4   Mauro  Idris       8
5    Fedo  Mauro       5
6   Mauro  Frago       7
7   Mauro  Frago       7
8   Frago  Mauro       6
9   Frago  Idris       9
10  Frago  Idris       9
11  Frago  Idris      10
12  Frago  Idris       8


In [4]:
winner_ids=torch.Tensor(winner_ids).int()
loser_ids=torch.Tensor(loser_ids).int()
loser_scores=torch.Tensor(loser_scores).int()

In [5]:
# Run inference using NUTS (No-U-Turn Sampler) in MCMC
nuts_kernel = NUTS(models.pyro_model)
mcmc_run = MCMC(nuts_kernel, num_samples=100, warmup_steps=10)
mcmc_run.run(winner_ids, loser_ids, loser_scores)

# Get posterior samples
posterior_samples = mcmc_run.get_samples()




Sample: 100%|██| 110/110 [02:47,  1.52s/it, step size=5.82e-05, acc. prob=0.974]


In [6]:
# Print the posterior mean and standard deviation of player strengths
for i in range(len(ids)):
    player = f'Player {i+1}: {dict_to_name[i]}'
    print(player)
    print("Posterior mean of strength:", posterior_samples['strength'][:, i].mean().item())
    print("Posterior std dev of strength:", posterior_samples['strength'][:, i].std().item())
    print()

Player 1: Fedo
Posterior mean of strength: 2.726062059402466
Posterior std dev of strength: 0.22691123187541962

Player 2: Mauro
Posterior mean of strength: -0.7610374689102173
Posterior std dev of strength: 0.17586161196231842

Player 3: Frago
Posterior mean of strength: 0.5311148166656494
Posterior std dev of strength: 0.10602498054504395

Player 4: Idris
Posterior mean of strength: 0.4968423545360565
Posterior std dev of strength: 0.04768441617488861



## Adding Gibbs sampling (or something similar)

In [8]:
winner_ids=winner_ids.numpy()
loser_ids=loser_ids.numpy()
loser_scores=loser_scores.numpy()

In [11]:
print(ids)

[0, 1, 2, 3]


In [85]:
'''def init_strength(nt):
    
    #init with prior
    init_strengths=np.random.uniform(0,1,nt)
    return init_strengths

def unnorm_posterior(x, mu, sigma, l1,lopps,n=11):
    #prior
    gaussian = norm.pdf(l1, mu, sigma)
    #likelihood
    p=l1/lopps
    negative_binomial = nbinom.pmf(x, n, p)
    return gaussian * negative_binomial

def eval_limits(strengths):
    #function that returns the extremes for the integration
    #these are given by the minimum and maximum among the current strength values
    minim=np.amin(strengths)
    maxim=np.amax(strengths)
    diff=maxim-minim
    return minim-3*diff, maxim+3*diff
    

def trap_integrator(n_steps=10):
    l1 = np.linspace(0.05,0.95, n_steps)
    p=(l1+epsilon)/(l1+lopps+2*epsilon)
    pmf_values = nbinom.pmf(x, 11,p)
    # use trapezoidal rule to approximate integral
    integral = trapz(pmf_values, p)
    return integral'''

In [215]:
print(strengths)
print(names)

[0.0870487  0.76039691 0.82454626 0.99754452]
['Fedo' 'Mauro' 'Frago' 'Idris']


In [ ]:
print()

In [193]:
'''def trap_integrator(n_steps=n_steps):
    l1 = np.linspace(0.05,0.95, n_steps)
    # use trapezoidal rule to approximate integral
    integral = trapz(product_function(l1), l1)
    return integral
def normalized_prod_function(l1):
    return unnorm_prod_function(l1)/trap_integrator()'''

### Choice: option 1

In [236]:
def unnorm_product_function(l1):
    if l1<0 or l1>1:
        return 0
    else:
        #commented stuff was useful for integration
        #l1=l1.reshape(len(l1),1)
        #print((l1/(l1+lambdas_wins)).shape)
        #print((lambdas_losses/(l1+lambdas_losses)).shape)
        #p=np.concatenate((l1/(l1+lambdas_wins),lambdas_losses/(l1+lambdas_losses)),axis=1)
        #print(p.shape)
        p=np.concatenate((l1/(l1+lambdas_wins),lambdas_losses/(l1+lambdas_losses)))
        #func=np.prod(nbinom.pmf(scores, 11,p),axis=1)
        func=np.prod(nbinom.pmf(scores, 11,p))
        #print(func)    
        return func

In [237]:
def slice_sampler(x_init=l1,custom_function=unnorm_product_function, n_samples=10):
    x = x_init
    samples = np.zeros(n_samples)
    for i in range(n_samples):
        # Draw a vertical line
        y = np.random.uniform(0, custom_function(x))
        # Create a horizontal “slice” (i.e., an interval)
        x_left = x - 1.
        while y < custom_function(x_left):
            x_left -= 1.
        x_right = x + 1.
        while y < custom_function(x_right):
            x_right += 1.
        # Draw new sample
        while True:
            x_new = np.random.uniform(x_left, x_right)
            if y < custom_function(x_new):
                break
            elif x_new > x:
                x_right = x_new
            elif x_new < x:
                x_left = x_new
            else:
                raise Exception("Slice sampler shrank to zero!")
        x = x_new
        samples[i] = x
    return samples[-1]

In [238]:
iterations=30
nt = len(ids)
n_steps=10
epsilon=0.001
strengths=init_strength(nt)
  

In [239]:
for i in range(iterations):
    random.shuffle(ids)
    print(i)
    for current_id in ids:
        l1=strengths[current_id]
        lambdas_wins=strengths[loser_ids[np.where(winner_ids == current_id)]]
        scores=loser_scores[np.where(winner_ids == current_id)]
        lambdas_losses=strengths[winner_ids[np.where(loser_ids == current_id)]]
        scores=np.concatenate((scores,loser_scores[np.where(loser_ids == current_id)]))
        strengths[current_id]=slice_sampler()

0


Exception: Slice sampler shrank to zero!

In [235]:
strengths

array([23.26738501, 11.87863302, 15.07578133, 11.30866744])